In [20]:
import pandas as pd
import numpy as np
from math import *
from src.display import horizontal_cluster
from src.find_HC import *
#from src.Project_step_1 import *

In [21]:
horizontal_cluster('AIQTSDEH...KV..ATPANW....RP..................GDK.V.VVP.PP.NTQE.MA..EERMKEG')

[['-' 'A' '▣' 'K' '★' 'R' 'K' '★' '☐' 'A' 'M']
 ['-' 'I' 'D' 'V' 'A' '★' 'V' '★' 'Q' 'E' 'K']
 ['-' 'Q' 'E' 'A' 'N' '⯁' 'V' '★' 'E' 'E' 'E']
 ['-' '☐' 'H' '☐' 'W' 'D' 'V' 'N' 'M' 'R' '⯁']
 ['A' '▣' 'K' '★' 'R' 'K' '★' '☐' 'A' 'M' '-']
 ['I' 'D' 'V' 'A' '★' 'V' '★' 'Q' 'E' 'K' '-']
 ['Q' 'E' 'A' 'N' '⯁' 'V' '★' 'E' 'E' 'E' '-']
 ['☐' 'H' '☐' 'W' 'D' 'V' 'N' 'M' 'R' '⯁' '-']]


In [127]:
# ça durera 1300 fois plus longtemps pour toute la database
Q_codes, all_HC = get_analyse('part_of_data.txt')

In [100]:
all_HC

{3: 383625,
 7: 95790,
 5: 293275,
 81: 24430,
 17: 139555,
 69: 18725,
 139: 5805,
 11: 97555,
 273: 15445,
 9: 234045,
 73: 44920,
 293: 13875,
 4689: 225,
 18757: 50,
 37: 49950,
 593: 2275,
 2373: 275,
 325: 3405,
 649: 5425,
 137: 30335,
 15: 26055,
 121: 4470,
 57: 14300,
 25: 67605,
 113: 13265,
 49: 38275,
 41: 38405,
 657: 2365,
 145: 26095,
 21: 66405,
 85: 22535,
 171: 6900,
 43: 15590,
 19: 50365,
 101: 9625,
 1301: 720,
 20817: 75,
 277: 2685,
 4433: 370,
 337: 6220,
 5193: 295,
 20773: 30,
 1097: 8255,
 4389: 420,
 585: 5475,
 203: 1530,
 407: 615,
 75: 10945,
 151: 3715,
 23: 17660,
 329: 6280,
 659: 1370,
 147: 9670,
 8867: 110,
 17735: 60,
 283761: 60,
 675: 2465,
 1351: 240,
 21617: 90,
 163: 9055,
 327: 1265,
 5233: 190,
 35: 47590,
 71: 4830,
 1137: 965,
 587: 5805,
 89: 25260,
 713: 3690,
 201: 16810,
 5257: 165,
 21029: 70,
 1161: 4420,
 4645: 1230,
 549: 4610,
 341: 12365,
 5205: 230,
 20821: 165,
 1109: 525,
 4437: 225,
 29: 18115,
 117: 1560,
 469: 615,
 13: 68

In [107]:
Q_codes[-1][len(Q_codes[-1])-100:]

[['DVU', 'MVM', 'MUVDVVMDD', 'MVMVVVMUMU', 'VU', 'VDVMVVUU'],
 ['D', 'VVVVM', 'D', 'V', 'UVD', 'V', 'VDUV', 'MDDVMVU'],
 ['MVMMUU', 'V', 'MU'],
 ['VMV', 'VVUUDVUUMVM', 'MM', 'MM'],
 ['D', 'M', 'VU', 'MVM', 'DVV', 'DUV', 'M', 'VM'],
 ['UU', 'VM', 'D', 'VVVUDV', 'MV', 'UVVUVDVV', 'VDUV', 'UDDVUMV'],
 ['MVUDU', 'VUM', 'V', 'DVUUV'],
 ['UUMVMVDDMVDM', 'VV', 'VDU'],
 ['DVM', 'MU', 'MV', 'MVMV', 'V'],
 ['VVU', 'V', 'M', 'UUVV', 'UDMVVVV', 'VM', 'VV'],
 ['UVU', 'VV', 'VU', 'MVMVDUUMDVV', 'UV', 'VUUVDU'],
 ['VVU', 'DV', 'MVV', 'MVV', 'DM', 'MU'],
 ['UV', 'VVVM', 'UV', 'MV', 'UVVUVDVV', 'VDUV', 'DVMVDU'],
 ['V', 'UDUMVU', 'DDUMVVD', 'M', 'VV', 'V', 'UV', 'UV'],
 ['DUUV', 'V', 'MVM', 'MV', 'VVUD', 'DVMVDD'],
 ['VUUMVUDUDDVV', 'V', 'VDU'],
 ['DDMVDDDV', 'MV', 'DV'],
 ['VV', 'D', 'VV', 'UVMDDM', 'MV', 'VDU', 'UVUUVDMVV'],
 ['VVU', 'MV', 'M', 'DVUMVV', 'DV', 'MVMV', 'D'],
 ['D', 'VMU', 'D', 'MV', 'UVD', 'M', 'VDUV', 'DDVUU'],
 ['MVM', 'UMMVDM', 'UM', 'UUMMVDUV'],
 ['DM', 'DVUMVVU', 'DVMDMVVV', 'UV'

In [22]:
structs, Q_code = binary_coding("AIQTSDEH...KV..ATPANW....RP..................GDK.V.VVP.PP.NTQE.MA..EERMKEG")

In [24]:
count_HC(structs, {})

{3: 2, 7: 1}

In [25]:
structs

['01', '1', '1', '111', '1']

In [26]:
Q_code

['VV']

In [113]:
a='-----....--...----.------...............---...---.---............-----....--.-....-.-..-.--.......--..-...-.-..-.-..-.-.-........................--.-.-...-.--...GQM............PRL..E..VV.Q....T..L.T..A.G...Y.....E....D....V....-L....P....L.L...P.D..........Q.....-....VT.......L............C......N.......A....A.......G.....L......H........D.....T...S....TA..ELAvalalasgrhldwfardqvtgtwgqrpgraladqrvlivgygrigaaiearlqgfevasvtrvsgrgrsgpplvhpvgdlhgllpe..................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................advvfvvaphtpqtegmfgarelgllpdgallvnvargklvdtaallaelsarritaaldvtdpeplpvghplwhapgvlisphvgGA..SSAFT.P..R.A.D.RLI.AAQLT.R.FA.AG......Q.PLANVV-'
a=a.replace('-', '')
a=a.replace('.', '')
print(step1(a))
print(step2(a))

⯁QM★RLEVVQ☐L☐A⯁YEDVL★LL★DQV☐LCNAA⯁LHD☐▣☐AELAvalalasgrhldwfardqvtgtwgqrpgraladqrvlivgygrigaaiearlqgfevasvtrvsgrgrsgpplvhpvgdlhgllpeadvvfvvaphtpqtegmfgarelgllpdgallvnvargklvdtaallaelsarritaaldvtdpeplpvghplwhapgvlisphvg⯁A▣▣AF☐★RADRLIAAQL☐RFAA⯁Q★LANVV
0010010110010001001101100010100000100000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000001100010010000010011


In [114]:
int('101100100010011', 2)

22803